# 弹性薄板（Kirchhoff板）

PDE:
    $$
    \frac{\partial ^4w}{\partial x^4} + \
    2\frac{\partial ^4w}{\partial x^2 \partial y^2} + \
    \frac{\partial ^4}{\partial y^4} = \frac{q}{D}
    $$
    $$
    D = \frac{E \epsilon^3}{12(1-\mu^2)}
    $$

其中，w为板的挠度，q为均布荷载大小，D为薄板抗弯刚度，E为弹性模量，delta为薄板厚度，mu为薄板泊松比

几何方程
    $$
    \begin{aligned}
    \epsilon_x &= -\frac{\partial^2w}{\partial x^2}z \\
    \epsilon_y &= -\frac{\partial^2w}{\partial y^2}z \\
    \gamma_{xy} &= -2\frac{\partial^2w}{\partial x \partial y}z
    \end{aligned}
    $$

本构方程：
    $$
    \begin{aligned}
    \sigma_x &= \frac{E}{1-\mu^2} \left( \epsilon_x + \mu \epsilon_y \right) \\
    \sigma_y &= \frac{E}{1-\mu^2} \left( \epsilon_y + \mu \epsilon_x \right) \\
    \tau_{xy} &= \frac{E}{2 \left( 1+\mu \right)} \gamma_{xy}
    \end{aligned}
    $$

## 1. Import and setup random seeds

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch import pi as pi
from torch import sin as sin
from torch import cos as cos
import matplotlib.pyplot as plt


def set_seeds(seeds):
    torch.manual_seed(seeds)
    torch.cuda.manual_seed(seeds)
    torch.backends.cudnn.deterministic = True

set_seeds(12)

g:\software\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Define MLP model

In [3]:
# input x, y, output u_x and u_y
class MLP(nn.Module):
    def __init__(self) -> None:
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(
            nn.Linear(2, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 1)
        )

    def forward(self, x):
        return self.model(x)

In [16]:
# l = torch.linspace(0, 1, 3)
# xm, ym = torch.meshgrid(l, l)
# xm = xm.reshape(-1, 1)
# ym = ym.reshape(-1, 1)
# print(xm)
# print(ym)

# xy = torch.concat([xm, ym], dim = 1)

# print(xy)
# print(xy.shape)

# model = MLP()

# y = model(xy)

# print(y)

sim_results = np.loadtxt("./assets/Plate.csv", delimiter=',')
node_id = sim_results[:, 0].reshape(-1, 1) # (400, 1)
node_x = sim_results[:, 1].reshape(-1, 1) # (400, 1)
node_y = sim_results[:, 2].reshape(-1, 1) # (400, 1)
node_w = sim_results[:, 3].reshape(-1, 1) # (400, 1)
node_sx = sim_results[:, 4].reshape(-1, 1) # (400, 1)
node_sy = sim_results[:, 5].reshape(-1, 1) # (400, 1)
node_sxy = sim_results[:, 6].reshape(-1, 1) # (400, 1)

node_sxy.shape

(400, 1)

In [17]:
torch.linspace(0, 1, 20)-0.5

tensor([-0.5000, -0.4474, -0.3947, -0.3421, -0.2895, -0.2368, -0.1842, -0.1316,
        -0.0789, -0.0263,  0.0263,  0.0789,  0.1316,  0.1842,  0.2368,  0.2895,
         0.3421,  0.3947,  0.4474,  0.5000])

## 3. Define loss function

In [ ]:
# material property, load condition, ABAQUS simulation results
E = 1000E9
mu = 0.25
q = 1E9
D = (E * 1.0 ** 3) / (12 * (1 - mu ** 2))

sim_results = np.loadtxt("./assets/Plate.csv", delimiter=',')
node_id = sim_results[:, 0].reshape(-1, 1) # (400, 1)
node_x = sim_results[:, 1].reshape(-1, 1) # (400, 1)
node_y = sim_results[:, 2].reshape(-1, 1) # (400, 1)
node_w = sim_results[:, 4].reshape(-1, 1) # (400, 1)
node_sx = sim_results[:, 5].reshape(-1, 1) # (400, 1)
node_sy = sim_results[:, 6].reshape(-1, 1) # (400, 1)
node_sxy = sim_results[:, 7].reshape(-1, 1) # (400, 1)


# mean square loss
mse_loss = nn.MSELoss()

# gradient
def gradient(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, only_inputs=True)[0]
    else:
        return gradient(gradient(u, x), x, order-1)


def loss_w(model, device):
    """
    ||w - w*||_2
    """

    global node_w

    x = torch.linspace(0, 1, 20, requires_grad=True, device=device)
    y = torch.linspace(0, 1, 20, requires_grad=True, device=device)

    xm, ym = torch.meshgrid(x, y)
    xm = xm.reshape(-1, 1)
    ym = ym.reshape(-1, 1)

    xm.requires_grad = True
    ym.requires_grad = True

    xy = torch.concat([xm, ym], dim = 1)
    w = model(xy)

    cond = torch.from_numpy(node_w)
    
    return mse_loss(w, cond)


def loss_pde(model, device):
    """
    ||nabla^4 w - q / D||_2
    """
    global q, D

    x = torch.linspace(0, 1, 20, requires_grad=True, device=device)
    y = torch.linspace(0, 1, 20, requires_grad=True, device=device)

    xm, ym = torch.meshgrid(x, y)
    xm = xm.reshape(-1, 1)
    ym = ym.reshape(-1, 1)

    xm.requires_grad = True
    ym.requires_grad = True

    xy = torch.concat([xm, ym], dim = 1)
    w = model(xy)

    nabla4w = gradient(w, x, 4) + gradient(w, y, 4) + 2 * gradient(gradient(x, 2), y, 2)

    cond = torch.ones_like(x, requires_grad=True, device=device) * q / D
    conda = cond.reshape(-1, 1)
    
    return mse_loss(nabla4w, cond)


def loss_sigma_x(model, device):
    """
    ||sigma_x - sigma_x*||_2
    """

    x = torch.linspace(0, 1, 20, requires_grad=True, device=device)
    y = torch.linspace(0, 1, 20, requires_grad=True, device=device)

    xm, ym = torch.meshgrid(x, y)
    xm = xm.reshape(-1, 1)
    ym = ym.reshape(-1, 1)

    xm.requires_grad = True
    ym.requires_grad = True

    xy = torch.concat([xm, ym], dim = 1)
    w = model(xy)

    ...

    cond = torch.zeros_like(x, requires_grad=True, device=device)
    
    return mse_loss(uy, cond)


def loss_uy_left(model, device):
    """
    u_y = 0
    """

    x = torch.linspace(0, 1, 20, device=device)
    y = torch.linspace(0, 1, 20, device=device)
    xm, ym = torch.meshgrid(x, y)
    xm.requires_grad = True
    ym.requires_grad = True

    uy = model(torch.cat([x, y], dim=1))[:, 1].view(-1, 1)
    cond = torch.zeros_like(x, requires_grad=True, device=device)
    
    return mse_loss(uy, cond)


## 4. Training

In [ ]:
# set training parameters
num_epochs = 1000
device = "cuda:0" if torch.cuda.is_available() else "cpu"

n_pde = 1000 # sampling nunber

# record loss
loss_list = []
best_loss = 1.0E10

# training
model = MLP().to(device=device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    optimizer.zero_grad()

    l00 = loss_ux_down(model, n_pde, device)
    l01 = loss_uy_down(model, n_pde, device)
    l02 = loss_uy_left(model, n_pde, device)
    l03 = loss_uy_right(model, n_pde, device)
    l04 = loss_ux_up(model, n_pde, device)
    l05 = loss_sxx_left(model, n_pde, device, la=la, mu=mu, q=q)
    l06 = loss_sxx_right(model, n_pde, device, la=la, mu=mu, q=q)
    l07 = loss_syy_up(model, n_pde, device, la=la, mu=mu, q=q)
    l08 = 0.01 * loss_fx(model, n_pde, device, la=la, mu=mu, q=q)
    l09 = 0.01 * loss_fy(model, n_pde, device, la=la, mu=mu, q=q)

    loss =  l00 + l01 + l02 + l03 + l04 + l05 + l06 + l07 + l08 + l09

    loss.backward()
    
    optimizer.step()

    if epoch % 100 == 0:
        # print("Epoch: ", epoch, " loss: ", loss.item())
        print("Epoch: ", epoch, end="\t")
        print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}".format(\
              l00.item(), l01.item(), l02.item(), l03.item(), l04.item(), l05.item(),
              l06.item(), l07.item(), l08.item(), l09.item(), loss.item()), sep='\t')

    # save model
    loss_list.append(loss.item()) # record loss
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), "./assets/plane_strain_linear_elastic_best.pth")


In [ ]:
# plot loss
plt.plot(np.array(loss_list))

## 5. Inference and visualization

In [ ]:
# material properties
la = 1
mu = 0.5
q = 4

# mesh
n_mesh = 101 # 画图网格密度

xc = torch.linspace(0, 1, n_mesh)
xm, ym = torch.meshgrid(xc, xc)

xm.requires_grad = True
ym.requires_grad = True

# real
ux_real = cos(2*pi*xm) * sin(pi*ym)
uy_real = sin(pi*xm) * q * ym**4 / 4

exx_real = gradient(ux_real, xm, 1) # epsilon xx
exy_real = 0.5 * (gradient(ux_real, ym, 1) + gradient(uy_real, xm, 1)) # epsilon xy
eyy_real = gradient(uy_real, ym, 1) # epsilon yy

sxx_real = 2 * mu * exx_real + la * (exx_real + eyy_real) # sigma xx
sxy_real = 2 * mu * exy_real # sigma xy
syy_real = 2 * mu * eyy_real + la * (exx_real + eyy_real) # sigma yy

# load model
model = MLP()
model.load_state_dict(torch.load("./assets/plane_strain_linear_elastic_best.pth"))

# prediction & error
xx = xm.reshape(-1, 1)
yy = ym.reshape(-1, 1)
xy = torch.cat([xx, yy], dim=1)

ux_pred = model(xy)[:, 0].reshape(xm.shape)
ux_error = torch.abs(ux_pred - ux_real)

uy_pred = model(xy)[:, 1].reshape(ym.shape)
uy_error = torch.abs(uy_pred - uy_real)

exx_pred = gradient(ux_pred, xx, 1).reshape(xm.shape)
exx_error = torch.abs(exx_pred - exx_real)

exy_pred = (0.5 * (gradient(ux_pred, yy, 1) + gradient(uy_pred, xx, 1))).reshape(xm.shape)
exy_error = torch.abs(exy_pred - exy_real)

eyy_pred = gradient(uy_pred, yy, 1).reshape(xm.shape)
eyy_error = torch.abs(eyy_pred - eyy_real)

sxx_pred = (2 * mu * exx_pred + la * (exx_pred + eyy_pred)).reshape(xm.shape)
sxx_error = torch.abs(sxx_pred - sxx_real)

sxy_pred = (2 * mu * exy_pred).reshape(xm.shape)
sxy_error = torch.abs(sxy_pred - sxy_real)

syy_pred = (2 * mu * eyy_pred + la * (exx_pred + eyy_pred)).reshape(xm.shape)
syy_error = torch.abs(syy_pred - syy_real)

# plot
xm = xm.detach().numpy()
ym = ym.detach().numpy()

ux_real = ux_real.detach().numpy()
ux_pred = ux_pred.detach().numpy()
ux_error = ux_error.detach().numpy()

uy_real = uy_real.detach().numpy()
uy_pred = uy_pred.detach().numpy()
uy_error = uy_error.detach().numpy()

exx_real = exx_real.detach().numpy()
exx_pred = exx_pred.detach().numpy()
exx_error = exx_error.detach().numpy()

exy_real = exy_real.detach().numpy()
exy_pred = exy_pred.detach().numpy()
exy_error = exy_error.detach().numpy()

eyy_real = eyy_real.detach().numpy()
eyy_pred = eyy_pred.detach().numpy()
eyy_error = eyy_error.detach().numpy()

sxx_real = sxx_real.detach().numpy()
sxx_pred = sxx_pred.detach().numpy()
sxx_error = sxx_error.detach().numpy()

sxy_real = sxy_real.detach().numpy()
sxy_pred = sxy_pred.detach().numpy()
sxy_error = sxy_error.detach().numpy()

syy_real = syy_real.detach().numpy()
syy_pred = syy_pred.detach().numpy()
syy_error = syy_error.detach().numpy()


In [ ]:
def my_subplot(x, y, *data):
    """
    len(data) == 3
    """

    assert(len(data) == 3)

    fig, axes = plt.subplots(1, 3, figsize=(15, 3.5))
    label_list = ["Pred", "Real", "Error"]

    for i in range(3):
        # plt.pcolormesh(xm, ym, ux_real, cmap=plt.cm.rainbow)
        # plt.pcolormesh(xm, ym, ux_real, cmap="RdBu_r", zorder=1)
        # plt.contour(xm, ym, ux_real, 50, cmap="RdBu_r", zorder=1)
        im = axes[i].pcolormesh(x, y, data[i], cmap=plt.cm.rainbow)
        axes[i].text(0.5, 0.9, label_list[i], transform=axes[i].transAxes)
        fig.colorbar(im, ax=axes[i])

    plt.show()

In [ ]:
# ux
my_subplot(xm, ym, ux_pred, ux_real, ux_error)

## References

